In [ ]:
# Utils for doing some logging on the results of the datasets (to be used later)


import json
from pathlib import Path


def get_unique_path(base_path: Path) -> Path:
    # return a unique file name
    if not base_path.exists():
        return base_path
    i = 1
    while True:
        new_path = base_path.with_name(f"{base_path.stem}_{i}{base_path.suffix}")
        if not new_path.exists():
            return new_path
        i += 1

def logging(results, dataset_name):
  # Save results to file
  output_dir = Path("outputs")
  output_dir.mkdir(parents=True, exist_ok=True)

  results_path = get_unique_path(output_dir / f"{dataset_name}_results.json")
  summary_path = get_unique_path(output_dir / f"{dataset_name}_summary.json")

  with open(results_path, "w") as f:
      json.dump(results, f, indent=2)

  # Compute and print accuracy
  correct = sum(r["reward"] for r in results if r["reward"] is not None)
  total = len(results)
  accuracy = correct / total if total > 0 else 0.0
  # Save summary to JSON

  summary = {
      "correct": correct,
      "total": total,
      "accuracy": accuracy
  }

  with open(summary_path, "w") as f:
      json.dump(summary, f, indent=2)
  print(f"\nAccuracy: {accuracy:.2%} ({correct}/{total})")

  print(f"Saved results to {results_path}")
  print(f"Saved summary to {summary_path}")


In [ ]:
# !pip install -q transformers torch accelerate sentencepiece
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 10.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

## **Model Loading & Function for Calculating**

In [ ]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset

MODEL_NAME = "Qwen/Qwen2-0.5B-Instruct"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MAX_LENGTH_GENERATION = 200 # Max tokens for rationale + answer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id


#def calculate_reward(final_answer, correct_answer):
#    """Calculates a binary reward based on correctness."""
#    if final_answer and final_answer == correct_answer:
#        return 1.0
#    return 0.0

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/659 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/988M [00:00<?, ?B/s]

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.


generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

In [ ]:
class TaskEvaluator:
    def parse_llm_output(self, output: str):
        raise NotImplementedError

    def format_question(self, question_data: dict):
        raise NotImplementedError

    def format_prompt(self, question: str):
        raise NotImplementedError

COMMON SENSE QA: Utils for Parsing Dataset for Input and Parsing the Outputs

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import requests, re

class CommonsenseQAEval(TaskEvaluator):
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.prompt_eos = self._load_prompt()

    def _load_prompt(self):
        url = "https://raw.githubusercontent.com/ezelikman/STaR/main/commonsenseqa/prompts.txt"
        prompt = requests.get(url).text
        prompt_eos = re.sub(r"\.\n\n", "." + self.tokenizer.eos_token + "\n\n", prompt)
        if not prompt.endswith(self.tokenizer.eos_token):
            prompt_eos = prompt_eos.rstrip() + self.tokenizer.eos_token
        return prompt_eos

    def format_question(self, question_data):
        question = question_data['question']
        choices_text = ""
        for i, choice_label in enumerate(question_data['choices']['label']):
            choice_text = question_data['choices']['text'][i]
            choices_text += f"({choice_label.lower()}) {choice_text}\n"

        return f"Q: {question}\nAnswer Choices:\n{choices_text.strip()}\nA: "

    def format_prompt(self, question_data):
        return f"{self.prompt_eos}\n\n{self.format_question(question_data)}The answer"

    def parse_llm_output(self, generated_text):
        rationale = generated_text
        final_answer = None

        final_answer = re.findall(r"\(([a-e])\)", generated_text, re.IGNORECASE)
        if final_answer:
            final_answer = final_answer[-1].lower()
            if len(final_answer) > 0:
                last_occurrence_index = generated_text.rfind(f"({final_answer})")
                if last_occurrence_index != -1:
                    rationale = generated_text[:last_occurrence_index].strip()
        return rationale, final_answer

In [ ]:
from tqdm import tqdm


import torch.nn.functional as F
from tqdm import tqdm

def generate_responses_generic(
    dataset,
    model,
    tokenizer,
    evaluator,
    device,
    max_tokens=200,
    reward_fn=None,
    verbose=True,
):
    """
    Generic function that setups up eval
    """
    results = []
    model.to(device)
    model.eval()

    print(f"The dataset has {len(dataset)} examples\n")

    for idx, item in enumerate(tqdm(dataset)):
        correct_answer = item.get('answerKey', '').lower()
        print(f"\nThis is example {idx + 1}\n")

        # Format input prompt using evaluator
        prompt = evaluator.format_prompt(item)
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

        # Generate model output
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                temperature=0.7,
                top_k=20,
                top_p=0.8,
            )

        # Extract generated text
        n_input_tokens = inputs.input_ids.shape[1]
        generated_ids = output_ids[0, n_input_tokens:]
        generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)

        # Compute log-probability of generated tokens
        logits = model(output_ids).logits[0, n_input_tokens:]
        log_probs_all = F.log_softmax(logits, dim=-1)
        token_log_probs = torch.gather(log_probs_all, 1, generated_ids.unsqueeze(1)).squeeze(1)
        total_log_prob = token_log_probs.sum().item()

        # Parse output
        rationale, final_answer = evaluator.parse_llm_output(generated_text)
        print()
        print("This is the rationale:\n", rationale)
        print("This is the final_answer:\n", final_answer)
        print("This is the correct_answer:\n", correct_answer)
        print("This was the prompt: \n", prompt)

        # Compute reward
        reward = reward_fn(final_answer, correct_answer) if reward_fn else None

        if verbose:
            print("\n--------------------")
            print(f"QUESTION:\n{evaluator.format_question(item)}")
            print(f"PROMPT:\n{prompt}")
            print(f"\nGENERATION:\n{generated_text}")
            print(f"\nPREDICTION: {final_answer}, TRUE ANSWER: {correct_answer}, REWARD: {reward}")
            print(f"logP(y | x) = {total_log_prob:.3f}")

        results.append({
            "question": evaluator.format_question(item),
            "prompt": prompt,
            "generated_text": generated_text,
            "final_answer": final_answer,
            "correct_answer": correct_answer,
            "rationale": rationale,
            "reward": reward,
            "logprob": total_log_prob,
        })

    return results


Run the Eval on CommonsenseQA

In [ ]:
import random

commonsense_qa_dataset = load_dataset("commonsense_qa", split=f'validation')

# subsample 100 examples to evaluate in the test split
commonsense_qa_dataset = commonsense_qa_dataset.select(random.sample(range(len(commonsense_qa_dataset)), 100))


import requests, re
url = "https://raw.githubusercontent.com/ezelikman/STaR/main/commonsenseqa/prompts.txt"
prompt = requests.get(url).text
prompt_eos = re.sub(r"\.\n\n", "." + tokenizer.eos_token + "\n\n", prompt)
if not prompt.endswith(tokenizer.eos_token):
    prompt_eos = prompt_eos.rstrip() +  tokenizer.eos_token

evaluator = CommonsenseQAEval(tokenizer)

def reward_fn(pred, gold):
    return int(pred == gold)

results = generate_responses_generic(
    dataset=commonsense_qa_dataset,
    model=model,
    tokenizer=tokenizer,
    evaluator=evaluator,
    device=DEVICE,
    reward_fn=reward_fn,
    verbose=False,
)

logging(results, "commonsense_qa")

README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

...


This is the rationale:
 must be an activity that students in engineering must do. Answer choices that relate to learning include:

(i) answering questions - Engineering involves solving problems and understanding complex systems.

(ii) learning language - Engineering deals with various aspects of technology such as mechanical design, electrical engineering, etc., which can be learned through reading books or studying online courses.

(iii) doing math - Engineering involves using mathematical principles and theories to solve problems.

(iv) beginning to study - This is not a requirement but could help a student develop skills like critical thinking and problem-solving.

Therefore, the correct answer is
This is the final_answer:
 c
This is the correct_answer:
 c
This was the prompt: 
 Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter
A: The answer must be used to absorb extra in

 99%|█████████▉| 99/100 [03:11<00:02,  2.32s/it]


This is the rationale:
 must be a process or action to change one’s life. Bettering oneself means improving one’s skills, knowledge, etc., while paying bills means making payments on debts. Eating chicken is not an action or process, but rather a dietary choice. Therefore, the answer is better yourself
This is the final_answer:
 b
This is the correct_answer:
 a
This was the prompt: 
 Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter
A: The answer must be used to absorb extra ink. Blotters are designed to absorb liquids. Therefore, the answer is blotter (e).<|im_end|>

Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk
A: The answer must require cable. Cable is used to provide satellite channels to televisions. Therefore, the answer is television (c).<|im_end|>

Q: The fox walked from the city 

100%|██████████| 100/100 [03:12<00:00,  1.93s/it]


This is the rationale:
 must be a place where one can store unused soap. Cans or containers are commonly found in households. Therefore, the answer is cabinet
This is the final_answer:
 a
This is the correct_answer:
 a
This was the prompt: 
 Q: What do people use to absorb extra ink from a fountain pen?
Answer Choices:
(a) shirt pocket
(b) calligrapher's hand
(c) inkwell
(d) desk drawer
(e) blotter
A: The answer must be used to absorb extra ink. Blotters are designed to absorb liquids. Therefore, the answer is blotter (e).<|im_end|>

Q: What home entertainment equipment requires cable?
Answer Choices:
(a) radio shack
(b) substation
(c) television
(d) cabinet
(e) desk
A: The answer must require cable. Cable is used to provide satellite channels to televisions. Therefore, the answer is television (c).<|im_end|>

Q: The fox walked from the city into the forest, what was it looking for?
Answer Choices:
(a) pretty flowers
(b) hen house
(c) natural habitat
(d) storybook
(e) dense forest
A: 

### String Editing Dataset requires more prepocessing since it's not available on Hugging Face

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
# some of this processing code was written with assistance / consultation from an LLM for cookie cutter pre-processing logic for a CSV.

import csv
import re

def strip_html(text):
    """Remove HTML tags like <span class='from'> and <span class='to'>."""
    return re.sub(r"<[^>]+>", "", text)

def load_turk_data(file_path):
    examples = []

    with open(file_path, newline='', encoding='utf-8') as csvfile:
        reader = csv.DictReader(csvfile)

        for row in reader:
            # Collect training input/output pairs
            train_pairs = []
            for i in range(8):
                before = strip_html(row[f'before_{i}'])
                after = strip_html(row[f'after_{i}'])
                train_pairs.append((before, after))

            # Collect test input/output
            test_before = strip_html(row['test_before'])
            test_after = strip_html(row['test_after'])

            # Collect hint if available
            hint = strip_html(row['hint']).strip()

            # Store everything
            examples.append({
                "example_pairs": train_pairs,
                "test_query": test_before,
                "answerKey": test_after,
                "hint": hint if hint else None
            })

    return examples



In [ ]:
first_set = load_turk_data('./turk_data.csv')
second_set = load_turk_data('./turk_data_2.csv')

string_editing_dataset = first_set + second_set
print(f"Loaded {len(string_editing_dataset)} examples.")

# Shuffle the dataset for random splitting
random.seed(21)
random.shuffle(string_editing_dataset)

split_ratio = 0.8  # 80% train, 20% test
split_index = int(len(string_editing_dataset) * split_ratio)

train_set = string_editing_dataset[:split_index]
test_set = string_editing_dataset[split_index:]

print(f"Train set: {len(train_set)} examples")
print(f"Test set: {len(test_set)} examples")

Loaded 2000 examples.
Train set: 1600 examples
Test set: 400 examples


In [ ]:
import random

sampled = random.sample(string_editing_dataset, 10)
for i, ex in enumerate(sampled, 1):
    print(f"\n--- Example {i} ---")
    print("Hint:", ex['hint'])
    print("Example pairs:")
    for before, after in ex['example_pairs']:
        print(f"  {before} → {after}")
    print("Test pair:")
    print(f"  {ex['example_pairs'][0]} → {ex['answerKey']}")


--- Example 1 ---
Hint: look at vowels
Example pairs:
  connections → cqfnnqfctqfqfns
  trellis → trqfllqfs
  morose → mqfrqfsqf
  tenacity → tqfnqfcqfty
  ploddings → plqfddqfngs
  shucked → shqfckqfd
  gazettes → gqfzqfttqfs
  theme → thqfmqf
Test pair:
  ('connections', 'cqfnnqfctqfqfns') → vqfqfwpqfqfnts

--- Example 2 ---
Hint: look at the beginning of the wordlook at consonants
Example pairs:
  proliferated → eroliferated
  lameness → eameness
  belongs → eelongs
  adore → adore
  pailfuls → eailfuls
  peeling → eeeling
  warning → earning
  noncommercials → eoncommercials
Test pair:
  ('proliferated', 'eroliferated') → euccotash

--- Example 3 ---
Hint: look at the beginning of the word
Example pairs:
  lynched → ljnched
  legumes → ljgumes
  replicating → ljplicating
  devilling → ljvilling
  nincompoop → ljncompoop
  hazelnuts → ljzelnuts
  uninstaller → ljinstaller
  fortissimo → ljrtissimo
Test pair:
  ('lynched', 'ljnched') → ljrplussing

--- Example 4 ---
Hint: look at th

# Processing the String Editing Dataset

In [ ]:
from typing import Optional
import random

class StringEditingEval(TaskEvaluator):
    def __init__(self, tokenizer, include_hint: bool = True):
        self.tokenizer = tokenizer
        self.include_hint = include_hint
        # Load from turk csvs
        #first_set = load_turk_data('./turk_data.csv')
        #second_set = load_turk_data('./turk_data_2.csv')
        #self.string_dataset = first_set + second_set
        self.prompt_eos = self.tokenizer.eos_token #self._load_prompt()

    #def _load_prompt(self):
    #    url = "https://raw.githubusercontent.com/ezelikman/STaR/main/commonsenseqa/prompts.txt"
    #    prompt = requests.get(url).text
    #    prompt_eos = re.sub(r"\.\n\n", "." + self.tokenizer.eos_token + "\n\n", prompt)
    #    if not prompt.endswith(self.tokenizer.eos_token):
    #        prompt_eos = prompt_eos.rstrip() + self.tokenizer.eos_token
    #    return prompt_eos

    # def format_prompt(self, example):
    #     """Construct the prompt including optional hint."""
    #     examples = "\n".join([f"{b} is edited to {a}{self.prompt_eos}" for b, a in example["example_pairs"]])
    #     hint = f"\nHint: {example['hint'].rstrip()}" if self.include_hint and example.get("hint") else ""
    #     string_instruction=f"""Find the rule applied to edit these example pairs. Apply this rule to the word {example['test_query'].rstrip()} and return the answer as a single word. Think carefully and only return a single word."""
    #     return f"\n{examples}\n{hint}\n{string_instruction}\nAnswer:"
    # def format_prompt(self, test_example):
    #     """
    #     Builds a few-shot prompt with n_shot examples from train_set, followed by the test query.
    #     """
    #     n_shot = 3
    #     # Randomly sample few-shot examples from the training set
    #     few_shot_examples = random.sample(train_set, min(n_shot, len(train_set)))

    #     # Construct few-shot examples block
    #     few_shot_blocks = []
    #     for ex in few_shot_examples:
    #         example_pairs = "\n".join([f"{b} is edited to {a}{self.prompt_eos}" for b, a in ex["example_pairs"]])
    #         hint = f"\nHint: {ex['hint'].rstrip()}" if self.include_hint and ex.get("hint") else ""
    #         instruction = f"Find the rule applied to edit these example pairs. Apply this rule to the word {ex['test_query'].rstrip()} and return the answer as a single word."
    #         few_shot_blocks.append(f"{example_pairs}{hint}\n{instruction}\nAnswer: {ex['answerKey']}")

    #     # Now build the test prompt
    #     test_example_pairs = "\n".join([f"{b} is edited to {a}{self.prompt_eos}" for b, a in test_example["example_pairs"]])
    #     hint = f"\nHint: {test_example['hint'].rstrip()}" if self.include_hint and test_example.get("hint") else ""
    #     instruction = f"Find the rule applied to edit these example pairs. Apply this rule to the word {test_example['test_query'].rstrip()} and return the answer as a single word."
    #     test_prompt = f"{test_example_pairs}{hint}\n{instruction}\nAnswer:"

    def format_prompt(self, example):
        """Construct the prompt including optional hint."""
        examples = "\n".join([f"{b} is edited to {a}{self.prompt_eos}" for b, a in example["example_pairs"]])
        hint = f"\nHint: {example['hint'].rstrip()}" if self.include_hint and example.get("hint") else ""
        string_instruction=f"""Find the rule applied to edit these example pairs. Apply this rule to the word {example['test_query'].rstrip()} and generate a rationale for the rule.""" #return the answer as a single word. Think step by step and generate a r"""
        return f"\n{examples}\n{hint}\n{string_instruction}\nAnswer:"

        # Combine everything into the final prompt
        # return "\n\n".join(few_shot_blocks + [test_prompt])

    def format_question(self, example):
        """Not needed for string editing."""
        return None

    def parse_llm_output(self, generated_text):
        """
        Return full output as rationale and the first token/string after 'Answer:' as final answer.
        """
        # print("this is the generated_text", generated_text)
        rationale = generated_text.strip()
        # Assume the answer is the first non-empty line of the output
        lines = [line.strip() for line in generated_text.strip().split('\n') if line.strip()]
        final_answer = lines[0] if lines else None
        return rationale, final_answer


In [ ]:
# Run eval on String Editing Dataset
import numpy as np

base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(DEVICE)
evaluator = StringEditingEval(tokenizer)
SEED = 21
random.seed(SEED)
torch.manual_seed(SEED)

# subsample k examples to evaluate in the test split
k = 10
subset_indices = random.sample(range(len(test_set)), k)
sample_string_editing = [string_editing_dataset[i] for i in subset_indices]

def reward_fn(pred, gold):
    return int(pred == gold)

results = generate_responses_generic(
    dataset=sample_string_editing,
    model=base_model,
    tokenizer=tokenizer,
    evaluator=evaluator,
    device=DEVICE,
    reward_fn=reward_fn,
    verbose=False,
)

logging(results, "string_editing")

The dataset has 10 examples



  0%|          | 0/10 [00:00<?, ?it/s]


This is example 1



 10%|█         | 1/10 [00:00<00:02,  3.08it/s]


This is the rationale:
 menstruation is edited to menstruate
This is the final_answer:
 menstruation is edited to menstruate
This is the correct_answer:
 menstruatsn
This was the prompt: 
 
explication is edited to explicatsn<|im_end|>
scolded is edited to scolded<|im_end|>
craved is edited to craved<|im_end|>
interrelations is edited to interrelatsns<|im_end|>
criers is edited to crsrs<|im_end|>
fallen is edited to fallen<|im_end|>
perambulator is edited to perambulator<|im_end|>
remarriage is edited to remarrsge<|im_end|>

Hint: look at vowels
Find the rule applied to edit these example pairs. Apply this rule to the word menstruation and generate a rationale for the rule.
Answer:

This is example 2



...


This is the rationale:
 The rule that can be used to edit these example pairs is "reformulate". This means to change or modify something in a more positive or constructive way, which makes it easier to understand and remember.

The reason for applying this rule to the word reinforcement is because it suggests making changes or modifying something in a more positive or constructive way. By doing so, we can make the word easier to understand and remember.
This is the final_answer:
 The rule that can be used to edit these example pairs is "reformulate". This means to change or modify something in a more positive or constructive way, which makes it easier to understand and remember.
This is the correct_answer:
 reilholhemelh
This was the prompt: 
 
winters is edited to wilhelh<|im_end|>
dreamer is edited to lheamer<|im_end|>
potentials is edited to potelhialh<|im_end|>
herrings is edited to helhilhs<|im_end|>
bouquets is edited to bouquelh<|im_end|>
conjugating is edited to colhugatilh<|i

 90%|█████████ | 9/10 [00:21<00:02,  2.49s/it]


This is the rationale:
 The rule is "blunting" applied to the word "bluntness". This means that when we apply the rule, the word becomes more blunt or harsher in tone than it was before.

So, the rationale for applying the rule is that if we apply the rule to "bluntness", the word will become more blunt or harsher in tone than it was before.
This is the final_answer:
 The rule is "blunting" applied to the word "bluntness". This means that when we apply the rule, the word becomes more blunt or harsher in tone than it was before.
This is the correct_answer:
 bluntinyg
This was the prompt: 
 
indexes is edited to indexes<|im_end|>
exonerating is edited to exoneratinyg<|im_end|>
disgruntled is edited to disygruntled<|im_end|>
unsigned is edited to unsiygned<|im_end|>
promiscuity is edited to promiscuity<|im_end|>
hollies is edited to hollies<|im_end|>
miaows is edited to miaows<|im_end|>
glimpses is edited to yglimpses<|im_end|>

Hint: (no hint for this problem)
Find the rule applied to e

100%|██████████| 10/10 [00:25<00:00,  2.51s/it]


This is the rationale:
 The rule used to edit "noise" is to remove all vowels, leaving only consonant sounds. This is because "noise" contains two vowels - /n/ and /ng/ which are not allowed in the rules we will apply to edit "noisy." Therefore, removing them from the original string results in "nois".
Rationale: The word "noise" has two vowels - /n/ and /ng/. Removing these vowels from the word removes all consonant sounds, resulting in "nois".
This is the final_answer:
 The rule used to edit "noise" is to remove all vowels, leaving only consonant sounds. This is because "noise" contains two vowels - /n/ and /ng/ which are not allowed in the rules we will apply to edit "noisy." Therefore, removing them from the original string results in "nois".
This is the correct_answer:
 bioise
This was the prompt: 
 
bushiness is edited to biushiness<|im_end|>
abolitionist is edited to abolitionist<|im_end|>
hertzes is edited to biertzes<|im_end|>
wusses is edited to biusses<|im_end|>
constituenc

In [ ]:
from tqdm import tqdm
import torch.nn.functional as F

def lm_score_rationale(model, tokenizer, item, generated_text, device, max_tokens=200):
    """
    Evaluate the generated rationale and answer using the LM as a reward model.
    Returns a scalar reward score.
    """
    rationale, answer = evaluator.parse_llm_output(generated_text)

    # Construct the full prompt for scoring
    prompt = evaluator.format_prompt(item)
    full_input = f"{prompt} {rationale}"  # Include rationale for scoring

    inputs = tokenizer(full_input, return_tensors="pt", truncation=True, max_length=1024).to(device)
    with torch.no_grad():
        outputs = model(**inputs,
                        max_new_tokens=max_tokens,
                        pad_token_id=tokenizer.pad_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        temperature=0.7,
                        top_k=20,
                        top_p=0.8)
        logits = outputs.logits

    # Score the generated text log-probability for the rationale
    target_ids = tokenizer(rationale, return_tensors="pt", truncation=True, max_length=1024).input_ids.to(device)
    target_len = target_ids.shape[1]
    logits = logits[:, -target_len:, :]
    log_probs = F.log_softmax(logits, dim=-1)
    log_probs_rationale = torch.gather(log_probs, 2, target_ids.unsqueeze(-1)).squeeze(-1).sum()

    return log_probs_rationale.item()

def get_optimizer(model, lr=3.5e-3):
  return torch.optim.AdamW(model.parameters(), lr=1e-5)

def train_with_rationale_rewards(dataset, model, tokenizer, evaluator, device, epochs=5, lr=1e-4):
    """
    Train arbitrary model with process-level (i.e. fine-grained) reward signal
    """
    model.train()
    optimizer = get_optimizer(model, lr=lr)

    losses =

    for epoch in tqdm(range(epochs)):
        print(f"\nEpoch {epoch + 1}")
        for item in tqdm(dataset, desc="Training", miniters=100):
            # Step 1: Sample (rationale, answer)
            prompt = evaluator.format_prompt(item)
            inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(device)

            output_ids = model.generate(
                **inputs,
                max_new_tokens=MAX_LENGTH_GENERATION,
                pad_token_id=tokenizer.pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                temperature=0.7,
                top_k=20,
                top_p=0.8,
            )

            n_input_tokens = inputs.input_ids.shape[1]
            generated_ids = output_ids[0, n_input_tokens:]
            generated_text = tokenizer.decode(generated_ids, skip_special_tokens=True)
            rationale, final_answer = evaluator.parse_llm_output(generated_text)

            # Step 2: Evaluate reward
            reward = lm_score_rationale(model, tokenizer, item, generated_text, device)

            # Step 3: Compute log-probs
            logits = model(output_ids).logits[0, n_input_tokens:]
            log_probs_all = F.log_softmax(logits, dim=-1)
            token_log_probs = torch.gather(log_probs_all, 1, generated_ids.unsqueeze(1)).squeeze(1)
            total_log_prob = token_log_probs.sum()

            # Step 4: Compute policy gradient loss
            loss = -reward * total_log_prob

            # Step 5: Update policy
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            optimizer.zero_grad()

            if hasattr(torch.cuda, "empty_cache"):
                torch.cuda.empty_cache()

In [ ]:
# Train base qwen model with fine-grained rewards on string editing
import random
evaluator = StringEditingEval(tokenizer)
train_string_editing = random.sample(string_editing_dataset, k=500)
train_with_rationale_rewards(train_string_editing, model, tokenizer, evaluator, DEVICE, epochs=1)

  0%|          | 0/1 [00:00<?, ?it/s]


Epoch 1



100%|██████████| 1/1 [55:49<00:00, 3349.34s/it]


In [ ]:
test_string_editing = [x for x in string_editing_dataset if x not in train_string_editing]

def reward_fn(pred, gold):
    return int(pred == gold)

# Randomly sample 100 test examples
results = generate_responses_generic(
    dataset=random.sample(test_string_editing, k=100),
    model=model,
    tokenizer=tokenizer,
    evaluator=evaluator,
    device=DEVICE,
    reward_fn=reward_fn,
    verbose=False
)


The dataset has 100 examples



  0%|          | 0/100 [00:00<?, ?it/s]


This is example 1



  1%|          | 1/100 [00:05<09:49,  5.95s/it]


This is example 2



...


This is example 99



 99%|█████████▉| 99/100 [10:07<00:06,  6.14s/it]


This is example 100



100%|██████████| 100/100 [10:13<00:00,  6.13s/it]


NameError: name 'logging' is not defined

In [ ]:
logging(results, "string_editing_rl")


Accuracy: 0.00% (0/100)
Saved results to outputs/string_editing_rl_results.json
Saved summary to outputs/string_editing_rl_summary.json


In [ ]:
commonsense_qa_dataset = load_dataset("commonsense_qa", split=f'validation')
len(commonsense_qa_dataset)

README.md:   0%|          | 0.00/7.39k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/1.25M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/160k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9741 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1221 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1140 [00:00<?, ? examples/s]

1221

In [ ]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    model.config.pad_token_id = model.config.eos_token_id

class CommonsenseQAParser:
    def __init__(self, tokenizer):
        self.tokenizer = tokenizer
        self.system_prompt = """You are an expert at applying commonsense reasoning to answer multiple-choice questions. You will be given a question with multiple answer choices, and you will be tasked with providing a brief rationale for your answer, followed by the correct answer choice. For example:

        Q: What do people use to absorb extra ink from a fountain pen?
        Answer Choices:
        (a) shirt pocket
        (b) calligrapher's hand
        (c) inkwell
        (d) desk drawer
        (e) blotter
        A: The answer must be used to absorb extra ink. Blotters are designed to absorb liquids. Therefore, the answer is blotter (e).

        Q: What home entertainment equipment requires cable?
        Answer Choices:
        (a) radio shack
        (b) substation
        (c) television
        (d) cabinet
        (e) desk
        A: The answer must require cable. Cable is used to provide satellite channels to televisions. Therefore, the answer is television (c).

        Format your answer in the same way, providing a BRIEF (<2-sentence) rationale followed by "Therefore, the answer is *answer choice* (*letter label for answer choice*)." Do not use any other format. If you are unsure, choose the most likely answer based on your reasoning.
        """

    def format_question(self, question_data):
        q = question_data['question']
        choices = "".join(f"({lbl.lower()}) {txt}\n"
            for lbl, txt in zip(
                question_data['choices']['label'], question_data['choices']['text']
            )
        )

        return f"Q: {q}\nAnswer Choices:\n{choices.strip()}\nA: "

    def format_prompt(self, question_data):
        messages = [
            {"role": "system", "content": self.system_prompt},
            {"role": "user", "content": self.format_question(question_data)}
        ]
        return tokenizer.apply_chat_template(
            messages, tokenize=False,
            add_generation_prompt=False, enable_thinking=False
        ), messages[-1]['content']

    def parse_llm_output(self, generated_text):
        rationale = generated_text.removeprefix("</think>").strip()
        matches = re.findall(r"\(([a-e])\)", generated_text, re.IGNORECASE)
        return rationale, (matches[-1].lower() if matches else None)

TEMP = 0.7
MAX_NEW_TOKENS = 200
BATCH_SIZE = 8
MAX_PROMPT_LEN = 1024

# REINFORCE UTILS
@torch.no_grad()
def sample_no_grad(prompt_ids, max_new_tokens=MAX_NEW_TOKENS, temp=TEMP):
    seq = model.generate(
        prompt_ids,
        max_new_tokens=max_new_tokens, temperature=temp, do_sample=True,
        eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id
    )
    return seq[:, prompt_ids.size(1):]

def compute_logprobs(prompt_ids, gen_ids, temp=TEMP):
    full_ids = torch.cat([prompt_ids, gen_ids], dim=1) # (B, T)
    full_logits = model(full_ids).logits / temp # (B, T, V)
    full_logprobs = F.log_softmax(full_logits, dim=-1) # (B, T, V)
    token_logprobs = full_logprobs[:, :-1, :].gather(2, full_ids[:, 1:].unsqueeze(-1)).squeeze(-1) # (B, T-1)
    return token_logprobs[:, prompt_ids.size(1)-1:].sum(dim=1) # (B,)

# REWARD UTILS
def compute_binary_reward(final_answer, correct_answer, question=None, rationale=None):
    return 1.0 if final_answer == correct_answer else 0.0

opt = torch.optim.AdamW(model.parameters(), lr=2e-5, betas=(0.9,0.999))

commonsense_qa_dataset = load_dataset("commonsense_qa", split=f'validation')
train_dataset = commonsense_qa_dataset
parser = CommonsenseQAParser(tokenizer)

for idx, item in enumerate(train_dataset):
    print("----"*20)
    print(f"\nEXAMPLE {idx + 1}:")

    # Format prompt and question
    prompt_str, formatted_question = parser.format_prompt(item)
    correct_answer = item.get('answerKey', '').lower()
    prompt_ids = tokenizer(prompt_str, return_tensors="pt", truncation=True, max_length=1024).input_ids.to(device)

    # Sample model output
    gen_ids = sample_no_grad(prompt_ids, max_new_tokens=MAX_NEW_TOKENS, temp=TEMP)
    gen_str = tokenizer.batch_decode(gen_ids, skip_special_tokens=True)[0]

    # Parse model output, compute reward and logprobs
    rationale, final_answer = parser.parse_llm_output(gen_str)
    logprob = compute_logprobs(prompt_ids, gen_ids, temp=TEMP)
    R = compute_binary_reward(correct_answer, final_answer, question=formatted_question, rationale=rationale)

    print("QUESTION:\n", formatted_question)
    print("RATIONALE:\n", rationale)
    print("FINAL ANSWER:", final_answer)
    print("CORRECT ANSWER:", correct_answer)
    print("REWARD:", R)
    print("LOGPROB:", logprob.item())
    loss = -(R * logprob).mean()
    opt.zero_grad()
    loss.backward()
    opt.step()

NameError: name 'tokenizer' is not defined